In [1]:
# How do you install Detectron2 using pip and check the version of Detectron2

#!pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-kgg8pg3z
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-kgg8pg3z
  Resolved https://github.com/facebookresearch/detectron2.git to commit b1c43ffbc995426a9a6b5c667730091a384e0fa4
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 66.0 MB/s eta 0:00:00
  Created wheel for detectron2: filename=detectron2-0.6-cp3

In [4]:
#Q2  How do you perform inference with Detectron2 using an online image

import requests
import cv2
import numpy as np
import matplotlib.pyplot as plt
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog


image_url = "/content/pexels-marx-ilagan-616381.jpg" 
response = cv2.imread(image_url)

cfg = get_cfg()
cfg.merge_from_file("detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  
cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"  
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  

predictor = DefaultPredictor(cfg)

outputs = predictor(image)


v = Visualizer(image[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
output_image = v.draw_instance_predictions(outputs["instances"].to("cpu"))

plt.figure(figsize=(10, 10))
plt.imshow(output_image.get_image()[:, :, ::-1])  
plt.axis('off')
plt.title('Detectron2 Inference Results')
plt.show()


AssertionError: Config file 'detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml' does not exist!

In [7]:
#Q2  How do you perform inference with Detectron2 using an online image

import requests
import cv2
import numpy as np
import matplotlib.pyplot as plt
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
import os


image_url = "/content/pexels-marx-ilagan-616381.jpg"  
image = cv2.imread(image_url)



cfg = get_cfg()
detectron2_path = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
config_file_path = os.path.join(detectron2_path, 'configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml')

cfg.merge_from_file(config_file_path)

cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"  
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  


predictor = DefaultPredictor(cfg)


outputs = predictor(image)


v = Visualizer(image[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
output_image = v.draw_instance_predictions(outputs["instances"].to("cpu"))


plt.figure(figsize=(10, 10))
plt.imshow(output_image.get_image()[:, :, ::-1])  
plt.axis('off')
plt.title('Detectron2 Inference Results')
plt.show()

In [8]:

#Q3 How do you visualize evaluation metrics in Detectron2, such as training loss

import os
import numpy as np
import matplotlib.pyplot as plt
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.evaluation import COCOEvaluator
from detectron2.utils.events import EventStorage

# Step 1: Setup configuration
cfg = get_cfg()
cfg.merge_from_file("detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  
cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"  
cfg.DATASETS.TRAIN = ("your_train_dataset_name",)  
cfg.DATASETS.TEST = ("your_val_dataset_name",)  
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  
cfg.SOLVER.MAX_ITER = 3000    
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128  
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  

class CocoTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        return COCOEvaluator(dataset_name, cfg, False, output_folder)


trainer = CocoTrainer(cfg)
trainer.resume_or_load(resume=False)


losses = []
with EventStorage() as storage:
    for iteration in range(cfg.SOLVER.MAX_ITER):
        trainer.run_step()  

        
        if iteration % 10 == 0:  
            loss_value = trainer.storage.history["loss"].latest()  
            losses.append(loss_value)
            storage.put_scalar("loss", loss_value)
            storage.step()


plt.figure(figsize=(10, 5))
plt.plot(range(len(losses)), losses, label='Training Loss')
plt.title('Training Loss Over Iterations')
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.legend()
plt.grid()
plt.show()


image_url = "th(1)eed.jpg" 
response = requests.get(image_url)
image = np.asarray(bytearray(response.content), dtype="uint8")
image = cv2.imdecode(image, cv2.IMREAD_COLOR)


predictor = DefaultPredictor(cfg)
outputs = predictor(image)

# Visualize the results on the online image
v = Visualizer(image[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
output_image = v.draw_instance_predictions(outputs["instances"].to("cpu"))

# Display the output image with bounding boxes and labels
plt.figure(figsize=(10, 10))
plt.imshow(output_image.get_image()[:, :, ::-1])  # Convert BGR to RGB for displaying
plt.axis('off')
plt.title('Detectron2 Inference Results')
plt.show()


In [ ]:
#Q4 How do you run inference with TFOD2 on an online image

import requests
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO


model_url = "http://download.tensorflow.org/models/faster_rcnn_resnet50_coco_2018_01_28.tar.gz"
model_dir = tf.keras.utils.get_file('faster_rcnn_resnet50_coco', model_url, untar=True)
model_dir = model_dir + "/frozen_inference_graph.pb"


detect_fn = tf.saved_model.load(model_dir)


def load_image(image_url):
    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content))
    image_np = np.array(image)
    return image_np

# Function to run inference on the image
def run_inference_for_single_image(model, image):
    input_tensor = tf.convert_to_tensor(image)
    input_tensor = input_tensor[tf.newaxis, ...]
    detections = model(input_tensor)
    return detections

def visualize_detections(image_np, detections):
    
    boxes = detections['detection_boxes'][0].numpy()
    classes = detections['detection_classes'][0].numpy().astype(np.int32)
    scores = detections['detection_scores'][0].numpy()

   
    threshold = 0.5
    for i in range(len(scores)):
        if scores[i] > threshold:
            box = boxes[i]
            (ymin, xmin, ymax, xmax) = box
            plt.gca().add_patch(plt.Rectangle((xmin * image_np.shape[1], ymin * image_np.shape[0]),
                                                (xmax - xmin) * image_np.shape[1],
                                                (ymax - ymin) * image_np.shape[0],
                                                fill=False, color='red', linewidth=2))
            plt.text(xmin * image_np.shape[1], ymin * image_np.shape[0],
                     f'Class: {classes[i]}, Score: {scores[i]:.2f}',
                     bbox=dict(facecolor='yellow', alpha=0.5), fontsize=12)


image_url = "https://upload.wikimedia.org/wikipedia/commons/6/60/Naxos_Taverna.jpg" 
image_np = load_image(image_url)


detections = run_inference_for_single_image(detect_fn, image_np)


plt.figure(figsize=(10, 10))
plt.imshow(image_np)
visualize_detections(image_np, detections)
plt.axis('off')
plt.title('Object Detection Results')
plt.show()



In [ ]:
#Q5 How do you install TensorFlow Object Detection API in Jupyter Notebook


!pip install tensorflow


!git clone https://github.com/tensorflow/models.git

!apt-get install -y protobuf-compiler  


%cd models/research
!protoc object_detection/protos/*.proto --python_out=.

!cp object_detection/packages/tf2/setup.py .
!python -m pip install .


!pip install cython
!pip install git+https://github.com/philferriere/cocoapi.git#subdirectory=PythonAPI

import object_detection
print("TensorFlow Object Detection API installed successfully.")



In [ ]:
#Q6 > How can you load a pre-trained TensorFlow Object Detection model

import tensorflow as tf
import numpy as np
import requests
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt

model_url = "http://download.tensorflow.org/models/object_detection/faster_rcnn_resnet50_coco_2017_11_17.tar.gz"
model_dir = tf.keras.utils.get_file('faster_rcnn_resnet50_coco', model_url, untar=True)


model_dir = model_dir + "/frozen_inference_graph.pb"
detect_fn = tf.saved_model.load(model_dir)


def load_image(image_url):
    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content))
    image_np = np.array(image)
    return image_np


def run_inference_for_single_image(model, image):
    input_tensor = tf.convert_to_tensor(image)
    input_tensor = input_tensor[tf.newaxis, ...] 
    detections = model(input_tensor)
    return detections

def visualize_detections(image_np, detections):
    boxes = detections['detection_boxes'][0].numpy()
    classes = detections['detection_classes'][0].numpy().astype(np.int32)
    scores = detections['detection_scores'][0].numpy()

    threshold = 0.5  
    for i in range(len(scores)):
        if scores[i] > threshold:
            box = boxes[i]
            (ymin, xmin, ymax, xmax) = box
            plt.gca().add_patch(plt.Rectangle((xmin * image_np.shape[1], ymin * image_np.shape[0]),
                                                (xmax - xmin) * image_np.shape[1],
                                                (ymax - ymin) * image_np.shape[0],
                                                fill=False, color='red', linewidth=2))
            plt.text(xmin * image_np.shape[1], ymin * image_np.shape[0],
                     f'Class: {classes[i]}, Score: {scores[i]:.2f}',
                     bbox=dict(facecolor='yellow', alpha=0.5), fontsize=12)


image_url = "https://upload.wikimedia.org/wikipedia/commons/6/60/Naxos_Taverna.jpg" 
image_np = load_image(image_url)


detections = run_inference_for_single_image(detect_fn, image_np)


plt.figure(figsize=(10, 10))
plt.imshow(image_np)
visualize_detections(image_np, detections)
plt.axis('off')
plt.title('Object Detection Results')
plt.show()


In [ ]:
#Q7 How do you preprocess an image from the web for TFOD2 inference

import numpy as np
import tensorflow as tf
import requests
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt


model_url = "http://download.tensorflow.org/models/faster_rcnn_resnet50_coco_2017_11_17.tar.gz"
model_dir = tf.keras.utils.get_file('faster_rcnn_resnet50_coco', model_url, untar=True)
model_dir = model_dir + "/frozen_inference_graph.pb"
detect_fn = tf.saved_model.load(model_dir)

def load_and_preprocess_image(image_url):
   
    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content))

    image = image.resize((640, 480))

    image_np = np.array(image) / 255.0 

    input_tensor = tf.convert_to_tensor(image_np)
    input_tensor = input_tensor[tf.newaxis, ...]

    return input_tensor

def run_inference_for_single_image(model, image):
    detections = model(image)
    return detections


def visualize_detections(image_np, detections):
    boxes = detections['detection_boxes'][0].numpy()
    classes = detections['detection_classes'][0].numpy().astype(np.int32)
    scores = detections['detection_scores'][0].numpy()

    threshold = 0.5  
    plt.figure(figsize=(10, 10))
    plt.imshow(image_np)

    for i in range(len(scores)):
        if scores[i] > threshold:
            box = boxes[i]
            (ymin, xmin, ymax, xmax) = box
            plt.gca().add_patch(plt.Rectangle((xmin * image_np.shape[1], ymin * image_np.shape[0]),
                                                (xmax - xmin) * image_np.shape[1],
                                                (ymax - ymin) * image_np.shape[0],
                                                fill=False, color='red', linewidth=2))
            plt.text(xmin * image_np.shape[1], ymin * image_np.shape[0],
                     f'Class: {classes[i]}, Score: {scores[i]:.2f}',
                     bbox=dict(facecolor='yellow', alpha=0.5), fontsize=12)


image_url = "https://upload.wikimedia.org/wikipedia/commons/6/60/Naxos_Taverna.jpg" 
input_tensor = load_and_preprocess_image(image_url)


detections = run_inference_for_single_image(detect_fn, input_tensor)


visualize_detections(np.array(Image.open(BytesIO(requests.get(image_url).content))), detections)
plt.axis('off')
plt.title('Object Detection Results')
plt.show()


In [ ]:
#Q8  How do you visualize bounding boxes for detected objects in TFOD2 inference

import numpy as np
import tensorflow as tf
import requests
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt


model_url = "http://download.tensorflow.org/models/faster_rcnn_resnet50_coco_2017_11_17.tar.gz"
model_dir = tf.keras.utils.get_file('faster_rcnn_resnet50_coco', model_url, untar=True)
model_dir = model_dir + "/frozen_inference_graph.pb"
detect_fn = tf.saved_model.load(model_dir)


def load_and_preprocess_image(image_url):
    
    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content))

    
    image_np = np.array(image) / 255.0 

    
    input_tensor = tf.convert_to_tensor(image_np)
    input_tensor = input_tensor[tf.newaxis, ...]  

    return input_tensor, image_np


def run_inference_for_single_image(model, image):
    detections = model(image)
    return detections


def visualize_detections(image_np, detections):
    boxes = detections['detection_boxes'][0].numpy()
    classes = detections['detection_classes'][0].numpy().astype(np.int32)
    scores = detections['detection_scores'][0].numpy()

    threshold = 0.5  
    plt.figure(figsize=(10, 10))
    plt.imshow(image_np)

    for i in range(len(scores)):
        if scores[i] > threshold:
            box = boxes[i]
            (ymin, xmin, ymax, xmax) = box
            plt.gca().add_patch(plt.Rectangle((xmin * image_np.shape[1], ymin * image_np.shape[0]),
                                                (xmax - xmin) * image_np.shape[1],
                                                (ymax - ymin) * image_np.shape[0],
                                                fill=False, color='red', linewidth=2))
            plt.text(xmin * image_np.shape[1], ymin * image_np.shape[0],
                     f'Class: {classes[i]}, Score: {scores[i]:.2f}',
                     bbox=dict(facecolor='yellow', alpha=0.5), fontsize=12)

image_url = "https://upload.wikimedia.org/wikipedia/commons/6/60/Naxos_Taverna.jpg"  
input_tensor, original_image_np = load_and_preprocess_image(image_url)


detections = run_inference_for_single_image(detect_fn, input_tensor)


visualize_detections(original_image_np, detections)
plt.axis('off')
plt.title('Object Detection Results')
plt.show()


In [ ]:
#Q9 How do you define classes for custom training in TFOD2

import tensorflow as tf

classes = ['class_name_1', 'class_name_2', 'class_name_3']
label_map_path = 'label_map.pbtxt'

with open(label_map_path, 'w') as f:
    for i, class_name in enumerate(classes, start=1):
        f.write(f"item {{\n  id: {i}\n  name: '{class_name}'\n}}\n")

config_path = 'models/research/object_detection/configs/tf2/faster_rcnn_resnet50_coco.config'
pipeline_config = tf.io.gfile.GFile(config_path, "r").read()
pipeline_config = pipeline_config.replace('num_classes: 90', f'num_classes: {len(classes)}')


with tf.io.gfile.GFile(config_path, "w") as f:
    f.write(pipeline_config)

print("Label map and configuration updated for custom training.")



In [ ]:
#Q10 How do you define classes for custom training in TFOD2

classes = ['class_name_1', 'class_name_2', 'class_name_3']
label_map_path = 'label_map.pbtxt'

with open(label_map_path, 'w') as f:
    for i, class_name in enumerate(classes, start=1):
        f.write(f"item {{\n  id: {i}\n  name: '{class_name}'\n}}\n")

config_path = 'models/research/object_detection/configs/tf2/faster_rcnn_resnet50_coco.config'
pipeline_config = tf.io.gfile.GFile(config_path, "r").read()
pipeline_config = pipeline_config.replace('num_classes: 90', f'num_classes: {len(classes)}')

with tf.io.gfile.GFile(config_path, "w") as f:
    f.write(pipeline_config)

print("Label map and configuration updated for custom training.")


In [ ]:
#Q11 How do you resize an image before detecting object1

import numpy as np
import tensorflow as tf
import requests
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt

model_url = "http://download.tensorflow.org/models/faster_rcnn_resnet50_coco_2017_11_17.tar.gz"
model_dir = tf.keras.utils.get_file('faster_rcnn_resnet50_coco', model_url, untar=True)
detect_fn = tf.saved_model.load(model_dir + "/frozen_inference_graph.pb")

def load_and_preprocess_image(image_url):
    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content))

    image = image.resize((640, 640))
    image_np = np.array(image) / 255.0  

    input_tensor = tf.convert_to_tensor(image_np)
    input_tensor = input_tensor[tf.newaxis, ...] 

    return input_tensor, image_np

def run_inference_for_single_image(model, image):
    detections = model(image)
    return detections

def visualize_detections(image_np, detections):
    boxes = detections['detection_boxes'][0].numpy()
    classes = detections['detection_classes'][0].numpy().astype(np.int32)
    scores = detections['detection_scores'][0].numpy()

    threshold = 0.5 
    plt.figure(figsize=(10, 10))
    plt.imshow(image_np)

    for i in range(len(scores)):
        if scores[i] > threshold:
            box = boxes[i]
            (ymin, xmin, ymax, xmax) = box
            plt.gca().add_patch(plt.Rectangle((xmin * image_np.shape[1], ymin * image_np.shape[0]),
                                                (xmax - xmin) * image_np.shape[1],
                                                (ymax - ymin) * image_np.shape[0],
                                                fill=False, color='red', linewidth=2))
            plt.text(xmin * image_np.shape[1], ymin * image_np.shape[0],
                     f'Class: {classes[i]}, Score: {scores[i]:.2f}',
                     bbox=dict(facecolor='yellow', alpha=0.5), fontsize=12)

image_url = "https://upload.wikimedia.org/wikipedia/commons/6/60/Naxos_Taverna.jpg"  
input_tensor, original_image_np = load_and_preprocess_image(image_url)

detections = run_inference_for_single_image(detect_fn, input_tensor)

visualize_detections(original_image_np, detections)
plt.axis('off')
plt.title('Object Detection Results')
plt.show()


In [ ]:
#Q12 > How can you apply a color filter (e.g., red filter) to an image?

from PIL import Image

image_path = 'path/to/your/image.jpg'
image = Image.open(image_path)

image = image.convert('RGB')

red_filter = Image.new('RGB', image.size, (255, 0, 0))  

filtered_image_pillow = Image.blend(image, red_filter, alpha=0.5)  # Adjust alpha for intensity

# Save or display the result using Pillow
filtered_image_pillow.show()  # Display the image
filtered_image_pillow.save('path/to/save/filtered_image_pillow.jpg')  # Save the filtered image

# Using OpenCV to apply a red filter
import cv2
import numpy as np

# Load the image using OpenCV
image_cv = cv2.imread(image_path)

# Create a red filter (BGR format)
red_filter_cv = np.zeros_like(image_cv)
red_filter_cv[:, :, 2] = 255  # Set the red channel to maximum

# Apply the red filter using OpenCV
filtered_image_cv = cv2.addWeighted(image_cv, 0.5, red_filter_cv, 0.5, 0)  # Adjust weights for intensity

# Display the result using OpenCV
cv2.imshow('Filtered Image (OpenCV)', filtered_image_cv)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Save the filtered image using OpenCV
cv2.imwrite('path/to/save/filtered_image_cv.jpg', filtered_image_cv)  # Save the filtered image
